In [1]:
import pandas as pd
import numpy as np
import regex as re
import pickle

In [2]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
%load_ext tensorboard

In [7]:
tensorboard --logdir "C:\\Users\\rpris\\Google Drive\\Project_2_DeepTextCorrector\\logs\\attention\\mono20210307-194817"

Reusing TensorBoard on port 6006 (pid 8964), started 0:01:29 ago. (Use '!kill 8964' to kill it.)

## Data Preprocessing

In [ ]:
data = open("/content/drive/MyDrive/Project_2/data/final_data_250k.txt")
#data = open("C:/Users/rpris/Google Drive/Project_2/data/final_data.txt")

text=[]
for f in data:
    f = re.sub(r"\n","",f)                 #removing /n

    if 'yer' in f.split():         #character hagrid from harry potter special language removed, his special english
        continue

    # specific
    f = re.sub(r"won't", "will not", f)              #TAKEN from assignment-20 text classification
    f = re.sub(r"can\'t", "can not", f)
    # general
    f = re.sub(r"n\'t", " not", f)
    f = re.sub(r"\'re", " are", f)
    f = re.sub(r"\'s", " is", f)
    f = re.sub(r"\'d", " would", f)
    f = re.sub(r"\'ll", " will", f)
    f = re.sub(r"\'t", " not", f)
    f = re.sub(r"\'ve", " have", f)
    f = re.sub(r"\'m", " am", f)
    
    f = re.sub(r"[^A-Za-z0-9\s]","", f )   #removing any special characters except ?
    f = f.lower()
    text.append(f)

    #break
data.close()

## Perturbations

In [ ]:
text[1111]

'harry was not sure he could explain'

In [6]:
from tqdm import tqdm
import random

In [ ]:
inp = []
out=[]
rand = [1,2,3]
for i in tqdm(text):
    temp = i.split()
    
    #removing a and also replacing with an (1 in every 5)
    if 'a' in temp:
        inp.append( (re.sub(r"\ba\b", "", i)).replace("  ", " ") )
        out.append(i)
        if random.sample(rand,1)[0] == 3:
            inp.append( re.sub(r"\ba\b", "an", i) )
            out.append(i)
            
    #removing an and also replacing with a (1 in every 5)
    if 'an' in temp:
        inp.append( (re.sub(r"\ban\b", "", i)).replace("  ", " ") )    #substituting and removing extra white space
        out.append(i)
        if random.sample(rand,1)[0] == 2:
            inp.append( re.sub(r"\ba\b", "an", i) )
            out.append(i)
            
    #removing the
    if 'the' in temp:
        inp.append( (re.sub(r"\bthe\b", "", i)).replace("  ", " ") )
        out.append(i)
    
    # replacing 'their' with 'there' and viceversa
    if 'there' in temp:
        inp.append( re.sub(r"\bthere\b","their", i) )
        out.append(i)
    
    if 'their' in temp:
        inp.append( re.sub(r"\btheir\b", "there", i) )
        out.append(i)
    
    #removing is
    if 'is' in temp:
        inp.append( (re.sub(r"\bis\b", "", i)).replace("  ", " ") )
        out.append(i)
    #removing are
    if 'are' in temp:
        inp.append( (re.sub(r"\bare\b", "", i)).replace("  ", " ") )
        out.append(i)
    #removing was
    if 'was' in temp:
        inp.append( (re.sub(r"\bwas\b", "", i)).replace("  ", " ") )
        out.append(i)
    #removing were
    if 'were' in temp:
        inp.append( (re.sub(r"\bwere\b", "", i)).replace("  ", " ") )
        out.append(i)
        
    #replacing than with then and vice versa
    if 'than' in temp:
        inp.append( re.sub(r"\bthan\b","then", i) )
        out.append(i)
    
    if 'then' in temp:
        inp.append( re.sub(r"\bthen\b", "than", i) )
        out.append(i)
    
    
    
    #replacing 'may' with 'would' randomly one of out 5 times
    if 'may' in temp:
        if random.sample(rand,1)[0] == 1:
            inp.append( re.sub(r"\bmay\b", "would", i) )
            out.append(i)
    
    #replacing he/she with 'they' randomly one out of five times
    if 'he' in temp:
        if random.sample(rand,1)[0] == 1:
            inp.append( re.sub(r"\bhe\b", "they", i) )
            out.append(i)
    if 'she' in temp:
        if random.sample(rand,1)[0] == 2:
            inp.append( re.sub(r"\bshe\b", "they", i) )
            out.append(i)

    

100%|██████████| 254195/254195 [00:02<00:00, 120672.17it/s]


In [ ]:
dataframe = pd.DataFrame({'encoder_input':inp, 'output':out})
print(dataframe.shape)
dataframe.head()

(207330, 2)


,encoder_input,output
0,mr dursley was the director of firm called gru...,mr dursley was the director of a firm called g...
1,mr dursley was director of a firm called grunn...,mr dursley was the director of a firm called g...
2,mr dursley the director of a firm called grunn...,mr dursley was the director of a firm called g...
3,they did not think they could bear it if anyon...,they did not think they could bear it if anyon...
4,none of them noticed large tawny owl flutter p...,none of them noticed a large tawny owl flutter...


### Examples random

In [ ]:
inp[0], out[0]

('mr dursley was the director of firm called grunnings which made drills',
 'mr dursley was the director of a firm called grunnings which made drills')

In [ ]:
inp[6700], out[6700]

('it is very dangerous to assume you know what woman wants',
 'it is very dangerous to assume you know what a woman wants')

In [ ]:
inp[3700], out[3700]

('they not birds', 'they are not birds')

## Input-Output Pair with teacher forcing

In [ ]:
dataframe['decoder_input'] = '<start> ' + dataframe['output'].astype(str)
dataframe['decoder_output'] = dataframe['output'].astype(str) + ' <end>'
dataframe = dataframe.drop(['output'], axis=1)

In [ ]:
dataframe.head()

,encoder_input,decoder_input,decoder_output
0,mr dursley was the director of firm called gru...,<start> mr dursley was the director of a firm ...,mr dursley was the director of a firm called g...
1,mr dursley was director of a firm called grunn...,<start> mr dursley was the director of a firm ...,mr dursley was the director of a firm called g...
2,mr dursley the director of a firm called grunn...,<start> mr dursley was the director of a firm ...,mr dursley was the director of a firm called g...
3,they did not think they could bear it if anyon...,<start> they did not think they could bear it ...,they did not think they could bear it if anyon...
4,none of them noticed large tawny owl flutter p...,<start> none of them noticed a large tawny owl...,none of them noticed a large tawny owl flutter...


### Example

In [ ]:
dataframe['encoder_input'][0]

'mr dursley was the director of firm called grunnings which made drills'

In [ ]:
dataframe['decoder_input'][0]

'<start> mr dursley was the director of a firm called grunnings which made drills'

In [ ]:
dataframe['decoder_output'][0]

'mr dursley was the director of a firm called grunnings which made drills <end>'

### train-test split

In [7]:
#train = pickle.load(open("C:/Users/rpris/Google Drive/Project_2/train.pkl", 'rb'))
train = pickle.load(open('/content/drive/MyDrive/Project_2/train-reg.pkl', 'rb'))

#validation = pickle.load(open("C:/Users/rpris/Google Drive/Project_2/validation.pkl", 'rb'))
validation = pickle.load(open('/content/drive/MyDrive/Project_2/validation-reg.pkl', 'rb'))

#test = pickle.load(open("C:/Users/rpris/Google Drive/Project_2/test.pkl", 'rb'))
test = pickle.load(open('/content/drive/MyDrive/Project_2/test-reg.pkl', 'rb'))

In [8]:
train.shape , validation.shape, test.shape

((123361, 3), (52869, 3), (31100, 3))

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataframe, test_size=0.15)
train, validation = train_test_split(train, test_size=0.3)
train.shape , validation.shape, test.shape

((123361, 3), (52869, 3), (31100, 3))

In [ ]:
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both decoder input and output
train.iloc[0]['decoder_input']= str(train.iloc[0]['decoder_input'])+' <end>'
train.iloc[0]['decoder_output']= str(train.iloc[0]['decoder_output'])+' <end>'

In [ ]:
import pickle
pickle.dump(train,open("/content/drive/MyDrive/Project_2/train-reg.pkl", "wb"))
pickle.dump(validation,open("/content/drive/MyDrive/Project_2/validation-reg.pkl", "wb"))
pickle.dump(test,open("/content/drive/MyDrive/Project_2/test-reg.pkl", "wb"))

# Model

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

## Tokenizing data

In [10]:
token_perturbation = pickle.load(open("/content/drive/MyDrive/Project_2/token_pertubation-reg.pkl",'rb'))
#token_perturbation = pickle.load(open("C:/Users/rpris/Google Drive/Project_2/token_pertubation.pkl", 'rb'))

token_correct = pickle.load(open("/content/drive/MyDrive/Project_2/token_correct-reg.pkl", 'rb'))
#token_correct = pickle.load(open("C:/Users/rpris/Google Drive/Project_2/token_correct.pkl", 'rb'))

In [ ]:
token_perturbation = Tokenizer(filters='"#$%&()*+-./:;=@[\\]^_`{|}~\t\n?,!')  # 
token_perturbation.fit_on_texts(train['encoder_input'].values)
token_correct = Tokenizer(filters='"#$%&()*+-./:;=@[\\]^_`{|}~\t\n?,!')   # 
token_correct.fit_on_texts(train['decoder_input'].values)

In [11]:
v_perturbation = len(token_perturbation.word_index.keys()) +1   # +1 not
print(v_perturbation)
v_correct=len(token_correct.word_index.keys())  + 1
print(v_correct)

28465
28467


In [ ]:
token_correct.word_index['<start>'] , token_correct.word_index['<end>']

(1, 18896)

In [ ]:
pickle.dump(token_perturbation,open("/content/drive/MyDrive/Project_2/token_pertubation-reg.pkl", "wb"))

In [ ]:
pickle.dump(token_correct, open("/content/drive/MyDrive/Project_2/token_correct-reg.pkl", "wb"))

## Creating embedding matrix from glove 300

In [12]:
embedding_matrix = pickle.load(open('/content/drive/MyDrive/Project_2/embedding_matrix.pkl','rb'))
embedding_matrix.shape

(28467, 300)

In [ ]:
embeddings_index = dict()
#f = open('C:\\Users\\rpris\\OneDrive\\Documents\\AI_data\\glove.6b\\glove.6B.300d.txt', encoding='utf8')
f = open("/content/drive/MyDrive/data_ml/glove.6B/glove.6B.300d.txt", encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((v_correct, 300))
for word, i in token_correct.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(28467, 300)

In [ ]:
pickle.dump(embedding_matrix, open('/content/drive/MyDrive/Project_2/embedding_matrix.pkl','wb'))

***

In [16]:
embedding_matrix_per = pickle.load(open('/content/drive/MyDrive/Project_2/embedding_matrix_per.pkl','rb'))
embedding_matrix_per.shape

(28465, 300)

In [14]:
embeddings_index = dict()
#f = open('C:\\Users\\rpris\\OneDrive\\Documents\\AI_data\\glove.6b\\glove.6B.300d.txt', encoding='utf8')
f = open("/content/drive/MyDrive/data_ml/glove.6B/glove.6B.300d.txt", encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix_per = np.zeros((v_perturbation, 300))
for word, i in token_perturbation.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_per[i] = embedding_vector
embedding_matrix_per.shape

(28465, 300)

In [15]:
pickle.dump(embedding_matrix_per, open('/content/drive/MyDrive/Project_2/embedding_matrix_per.pkl','wb'))

# Vanilla Encoder-Decoder

## Creating architecture using model subclassing

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, enc_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.enc_units= enc_units
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
    def build(self, input_shape):
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder", input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
        
    def call(self, input_sentances, training=True):
        input_embedd                        = self.embedding(input_sentances)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c
    def get_states(self):
        return self.lstm_state_h,self.lstm_state_c


In [ ]:
encoder_vanilla = pickle.dump(Encoder, open('/content/drive/MyDrive/Project_2/encodv.pkl','wb'))

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, dec_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.input_length = input_length
    
    def build(self, input_shape):
        # we are using embedding_matrix weights and not training the embedding layer
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", weights=[embedding_matrix],input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Encoder_LSTM")
        
    def call(self, target_sentances, state_h, state_c):
        target_embedd           = self.embedding(target_sentances)
        lstm_output, _,_        = self.lstm(target_embedd, initial_state=[state_h, state_c])
        return lstm_output

## Data pipeline

In [17]:
class Dataset:
    def __init__(self, data, token_perturbation, token_correct, max_len):
        self.encoder_inps = data['encoder_input'].values
        self.decoder_inps = data['decoder_input'].values
        self.decoder_outs = data['decoder_output'].values
        self.token_correct = token_correct
        self.token_perturbation = token_perturbation
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.token_perturbation.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.token_correct.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.token_correct.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)



In [18]:
class Dataloader(tf.keras.utils.Sequence):
    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        
        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

## Model

In [ ]:
class vanilla_model(Model):
    def __init__(self, encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__() 
        self.encoder = Encoder(vocab_size=v_perturbation, embedding_dim=300, input_length=encoder_inputs_length, enc_units=256)
        self.decoder = Decoder(vocab_size=v_correct, embedding_dim=300, input_length=decoder_inputs_length, dec_units=256)
        self.dense   = Dense(output_vocab_size, activation='softmax')
        
        
    def call(self, data):
        input,output = data[0], data[1]
        encoder_output, encoder_h, encoder_c = self.encoder(input)
        decoder_output                       = self.decoder(output, encoder_h, encoder_c)
        output                               = self.dense(decoder_output)
        return output

In [ ]:
train_dataset = Dataset(train, token_perturbation, token_correct, 16)
validation_dataset  = Dataset(validation, token_perturbation, token_correct, 16)
test_dataset = Dataset(test, token_perturbation, token_correct, 16)

train_dataloader = Dataloader(train_dataset, batch_size=512)
validation_dataloader = Dataloader(validation_dataset, batch_size=512)
test_dataloader = Dataloader(test_dataset, batch_size=512)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 16) (512, 16) (512, 16)


In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
vanilla = vanilla_model(encoder_inputs_length=16,decoder_inputs_length=16,output_vocab_size=v_correct)
optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)
vanilla.compile(optimizer= optimizer, loss= loss_function, metrics=['accuracy'])

In [ ]:
import datetime

logfile = "/content/drive/MyDrive/Project_2/logs/vanilla/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#logfile = "C:\\Users\\rpris\\Google Drive\\Project_2\\logs\\vanilla\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tfboard = tf.keras.callbacks.TensorBoard(log_dir=logfile, histogram_freq=1, write_graph=True)

chkfile = "/content/drive/MyDrive/Project_2/wts/vanilla/weights-{epoch:02d}-{val_loss:.4f}.hdf5"
#chkfile2 = "C:\\Users\\rpris\\Google Drive\\Project_2\\wts\\vanilla\\weights-{epoch:02d}-{val_loss:.4f}.hdf5"
chkpt = tf.keras.callbacks.ModelCheckpoint(chkfile, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=0, mode='min')

stp = tf.keras.callbacks.EarlyStopping(patience=7, monitor='val_loss')

In [ ]:
vanilla.load_weights("/content/drive/MyDrive/Project_2/wts/vanilla/weights-65-0.5898.hdf5")

In [ ]:
vanilla.summary()

Model: "vanilla_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Encoder)            multiple                  8705768   
_________________________________________________________________
decoder (Decoder)            multiple                  8705168   
_________________________________________________________________
dense (Dense)                multiple                  6968812   
Total params: 24,379,748
Trainable params: 16,244,948
Non-trainable params: 8,134,800
_________________________________________________________________


In [ ]:
train_steps=train.shape[0]//512
valid_steps=validation.shape[0]//512
vanilla.fit(train_dataloader, steps_per_epoch=train_steps, epochs=100, validation_data=validation_dataloader,
                      validation_steps=valid_steps, callbacks=[stp, chkpt, tfboard], initial_epoch=57)

Epoch 58/100
115/115 [==============================] - 46s 397ms/step - loss: 0.1084 - accuracy: 0.9452 - val_loss: 0.6000 - val_accuracy: 0.7373
Epoch 59/100
115/115 [==============================] - 44s 382ms/step - loss: 0.1012 - accuracy: 0.9502 - val_loss: 0.5956 - val_accuracy: 0.7411
Epoch 60/100
115/115 [==============================] - 43s 377ms/step - loss: 0.0970 - accuracy: 0.9526 - val_loss: 0.5958 - val_accuracy: 0.7404
Epoch 61/100
115/115 [==============================] - 43s 378ms/step - loss: 0.0936 - accuracy: 0.9543 - val_loss: 0.5913 - val_accuracy: 0.7428
Epoch 62/100
115/115 [==============================] - 44s 380ms/step - loss: 0.0892 - accuracy: 0.9568 - val_loss: 0.5916 - val_accuracy: 0.7456
Epoch 63/100
115/115 [==============================] - 43s 378ms/step - loss: 0.0844 - accuracy: 0.9597 - val_loss: 0.5918 - val_accuracy: 0.7461
Epoch 64/100
115/115 [==============================] - 43s 377ms/step - loss: 0.0814 - accuracy: 0.9610 - val_loss: 0

### Logs

#### After epoch 65, the model reaches its minimum loss

#### Orange- Train
#### Blue- Validation

<img src ="https://imgur.com/assAhwy.png">
<img src ="https://imgur.com/hkMReZX.png" >

## Inference

In [ ]:
def inference(enc_inp,dec_inp):
    
    translation=""

    e_input=[]
    for i in enc_inp.split():
        if token_perturbation.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(token_perturbation.word_index.get(i))
    
    #e_input = pad_sequences(e_input, maxlen=16, padding='post')
    
    
    e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))
    
    #there is no onestep decoder in this thing, so I have to use the decoder input to predict output
    
    #decoder input
    d_input=[]
    for i in dec_inp.split():
        if token_correct.word_index.get(i) == None:
            d_input.append(0)
        else:
            d_input.append(token_correct.word_index.get(i))

    #d_input = pad_sequences(d_input, maxlen=16, padding='post')
    
    prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

    for word in prediction[0]:
        word = token_correct.index_word[tf.argmax(word).numpy()]
        if word == "<end>":
            return translation
        translation += word + " "
    
    return translation
    

In [ ]:
sample_train = train.sample(1000)
result = []
i=0
for enc_inp,dec_inp,_ in sample_train.values:
    pred = inference(enc_inp,dec_inp)
    result.append(pred)

sample_train['correct_output'] = sample_train['decoder_output']
sample_train['predicted_output'] = result

sample_train = sample_train.drop(['decoder_input', 'decoder_output'], axis=1)

sample_train.head(10)

,encoder_input,correct_output,predicted_output
45193,it was 25 signs he great in bed,it was 25 signs he is great in bed <end>,it was 25 signs he is great in bed
56974,we good?,are we good? <end>,are we good?
78004,"look, you said you not feeling great","look, you said you were not feeling great <end>","look, you said you were not feeling great"
20015,what his name,what was his name <end>,what is his name
15078,let is head out to tarmac,let is head out to the tarmac <end>,let is head out to the tarmac
1170,"and this ron, our brother","and this is ron, our brother <end>","and this is ron, our brother"
32331,he after me,he is after me <end>,he is after me
76570,the important thing not to panic,the important thing is not to panic <end>,the important thing is not to panic
80471,i do not know how many other relatives he got ...,i do not know how many other relatives he is g...,i do not know how many other relatives he is g...
40691,"quot well i believe in god, and only thing tha...","quot well i believe in god, and the only thing...","well i i believe in god, the the only thing th..."


In [ ]:
sample_validation = validation.sample(1000)
result = []
for enc_inp,dec_inp,_ in sample_validation.values:
    pred = inference(enc_inp,dec_inp)
    result.append(pred)

sample_validation['correct_output'] = sample_validation['decoder_output']
sample_validation['predicted_output'] = result
sample_validation = sample_validation.drop(['decoder_input', 'decoder_output'], axis=1)
sample_validation.head(10)

,encoder_input,correct_output,predicted_output
45485,"well, it not exactly the term i would have cho...","well, it is not exactly the term i would have ...","it it is not sick, i me, i can have as as as c..."
43127,"you need anything, help of any kind, gimme holler","you need anything, help of any kind, gimme a h...","you need anything, help any any kind, gimme a ..."
58875,tasting and enjoying life the only thing of va...,tasting and enjoying life is the only thing of...,and and the the were the only our must leavin ...
71081,why you carrying the dog?,why are you carrying the dog? <end>,why are you carrying the dog?
48388,"hey, angela, sal and i go back long way","hey, angela, sal and i go back a long way <end>","yeah, thank i i just go back down back way"
72697,that is what you giving me?,that is what you are giving me? <end>,that is what you were giving me?
47689,"what is code, travers?","what is the code, travers? <end>","what is the code, travers?"
76278,someone names an lot of warts on line two,someone names a lot of warts on line two <end>,someone a a lot of the the the the
26049,if another ship had been beneath us klingons w...,if another ship had been beneath us the klingo...,"if a a is a locked out, would would could neve..."
73197,"what the problem, molly?","what is the problem, molly? <end>","what is the problem, molly?"


In [ ]:
sample_test = test.sample(1000)
result = []
for enc_inp,dec_inp,_ in sample_test.values:
    pred = inference(enc_inp,dec_inp)
    result.append(pred)
sample_test['correct_output'] = sample_test['decoder_output']
sample_test['predicted_output'] = result
sample_test = sample_test.drop(['decoder_input', 'decoder_output'], axis=1)
sample_test.head(10)

,encoder_input,correct_output,predicted_output
77691,we in the spite,we are in the spite <end>,we are in a waste
33526,"he said, yes, doctor","he said, yes, the doctor <end>","he said, yes, the doctor"
59705,you will probably be richest man in america so...,you will probably be the richest man in americ...,you will probably be a most in in helping to t...
77919,i will be right their,i will be right there <end>,i will be right there
42966,"she a little nuts, but she a good girl i think","she is a little nuts, but she is a good girl i...","she is a little nuts, but she is a good girl i..."
67483,this girl a godsend,this girl is a godsend <end>,this girl is a godsend
12904,you are right,you were right <end>,you were right
2450,there a note!,there is a note! <end>,there is a note!
37161,it perhaps strange that we both should be in l...,it is perhaps strange that we both should be i...,"it is perhaps strange than we can you, be inte..."
11335,"that you, jeffrey?","is that you, jeffrey? <end>","is that you, jeffrey?"


## BLEU Score

In [ ]:
import nltk.translate.bleu_score as bleu

In [ ]:
train_bleu = []
for enc_inp,correct,pred in sample_train.values:
    
    correct = correct.split()[:-1]    #removing end
    pred = pred.split()
    if len(correct) == len(pred):
        train_bleu.append(bleu.sentence_bleu([correct],pred))
print("BLEU Score of train dataset is",sum(train_bleu)/len(train_bleu))

BLEU Score of train dataset is 0.9444792707470997


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


<font size=4 color='red'> BLEU Score of train dataset is 0.9444792707470997

In [ ]:
validation_bleu = []
for enc_inp,correct,pred in sample_validation.values:
    
    correct = correct.split()[:-1]    #removing end
    pred = pred.split()
    if len(correct) == len(pred):
        validation_bleu.append(bleu.sentence_bleu([correct],pred))
print("BLEU Score of validation dataset is",sum(validation_bleu)/len(validation_bleu))

BLEU Score of validation dataset is 0.7513216789262608


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


<font size=4 color='red'> BLEU Score of validation dataset is 0.7513216789262608

In [ ]:
test_bleu = []
for enc_inp,correct,pred in sample_test.values:
    
    correct = correct.split()[:-1]    #removing end
    pred = pred.split()
    if len(correct) == len(pred):
        test_bleu.append(bleu.sentence_bleu([correct],pred))
print("BLEU Score of test dataset is",sum(test_bleu)/len(test_bleu))

BLEU Score of test dataset is 0.7486602027685026


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


<font size=4 color='red'> BLEU Score of test dataset is 0.7486602027685026

# Attention Mechanism

In [19]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints

## Architecture

### Encoder

In [ ]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):
        super().__init__()

        self.inp_vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.lstm_size = lstm_size
        self.input_length = input_length

    
    #def build(self, input_shape):
        self.embedd = Embedding(input_dim = self.inp_vocab_size, output_dim = self.embedding_size, input_length=self.input_length,
                                weights = [embedding_matrix_per], mask_zero=True)
        
        self.encod_lstm = LSTM(units = self.lstm_size, return_sequences=True, return_state=True, 
                               name="Encoder_Attention", activation = 'tanh',recurrent_activation ='sigmoid', 
                               recurrent_dropout = 0, unroll=False, use_bias=True, 
                               kernel_regularizer= regularizers.l2(1e-6))

    def call(self,input_sequence, training=True):
        embeddings = self.embedd(input_sequence)

        encod_out, encod_h, encod_c = self.encod_lstm(embeddings)

        return encod_out, encod_h, encod_c

    def initialize_states(self,batch_size):

        hidden = np.zeros((batch_size, self.lstm_units))

        cell = np.zeros((batch_size, self.lstm_units))
        
        return hidden, cell

    def get_config(self):
        '''Config'''

        config = { 'inp_vocab_size': self.inp_vocab_size,
                  'embedding_size': self.embedding_size,
                  'lstm_size': self.lstm_size,
                  'input_length': self.input_length
        }

        base_config = super(Encoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)

### Attention

https://github.com/UdiBhaskar/TfKeras-Custom-Layers/blob/master/Seq2Seq/clayers.py

#### Monotonic Bahanau Attention

In [20]:
def _attention_score(dec_ht,
                     enc_hs,
                     attention_type,
                     weightwa=None,
                     weightua=None,
                     weightva=None):
    if attention_type == 'bahdanau':
        score = weightva(tf.nn.tanh(weightwa(dec_ht) + weightua(enc_hs)))
        score = tf.squeeze(score, [2])
    elif attention_type == 'dot':
        score = tf.matmul(dec_ht, enc_hs, transpose_b=True)
        score = tf.squeeze(score, 1)
    elif attention_type == 'general':
        score = weightwa(enc_hs)
        score = tf.matmul(dec_ht, score, transpose_b=True)
        score = tf.squeeze(score, 1)
    elif attention_type == 'concat':
        dec_ht = tf.tile(dec_ht, [1, enc_hs.shape[1], 1])
        score = weightva(tf.nn.tanh(weightwa(tf.concat((dec_ht, enc_hs), axis=-1))))
        score = tf.squeeze(score, 2)
    return score


In [21]:
def _monotonic_attetion(probabilities, attention_prev, mode):

    if mode == 'hard':
        #Remove any probabilities before the index chosen last time step
        probabilities = probabilities*tf.cumsum(attention_prev, axis=1)
        attention = probabilities*tf.math.cumprod(1-probabilities, axis=1, exclusive=True)
    
    elif mode == 'parallel':
        cumprod_1mp_probabilities = tf.exp(tf.cumsum(tf.math.log(tf.clip_by_value(1-probabilities,\
            1e-10, 1)), axis=1, exclusive=True))
        attention = probabilities*cumprod_1mp_probabilities*tf.cumsum(attention_prev/\
            tf.clip_by_value(cumprod_1mp_probabilities, 1e-10, 1.), axis=1)
    else:
        raise ValueError("Mode must be 'hard', 'parallel' or 'recursive' ")

    return attention


In [ ]:
class MonotonicBahaAtt(tf.keras.layers.Layer):
    def __init__(self, units,
                 mode,
                 return_aweights=False,
                 scaling_factor=None,
                 noise_std=0,
                 weights_initializer='he_normal',
                 bias_initializer='zeros',
                 **kwargs):
        
        if 'name' not in kwargs:
            kwargs['name'] = ""
            
        super(MonotonicBahaAtt, self).__init__(**kwargs)
        self.units = units
        self.mode = mode
        self.scaling_factor = scaling_factor
        self.noise_std = noise_std
        self.weights_initializer = initializers.get(weights_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.weights_regularizer = regularizers.l2(1e-2)
    
    def build(self, input_shape):
        self._wa = layers.Dense(self.units, use_bias=False,\
            kernel_initializer=self.weights_initializer, bias_initializer=self.bias_initializer,\
                kernel_regularizer= self.weights_regularizer, name=self.name+"Wa")
        
        self._ua = layers.Dense(self.units,\
            kernel_initializer=self.weights_initializer, bias_initializer=self.bias_initializer,\
                kernel_regularizer= self.weights_regularizer, name=self.name+"Ua")
        
        self._va = layers.Dense(1, use_bias=False, kernel_initializer=self.weights_initializer,\
            kernel_regularizer= self.weights_regularizer,bias_initializer=self.bias_initializer, name=self.name+"Va")
        
        
    def call(self, decoder_hidden_state, encoder_outputs, prev_attention, training=True):

        encoder_outputs, decoder_hidden_state = tf.cast(encoder_outputs, tf.float32), \
            tf.cast(decoder_hidden_state, tf.float32)
        
        dec_hidden_with_time_axis = tf.expand_dims(decoder_hidden_state, 1)

        # score shape == (batch_size, max_length)
        score = _attention_score(dec_ht=dec_hidden_with_time_axis, enc_hs=encoder_outputs,\
                    attention_type='bahdanau', weightwa=self._wa,\
                        weightua=self._ua, weightva=self._va)
        
        if self.scaling_factor is not None:
            score = score/tf.sqrt(self.scaling_factor)

        if training:
            if self.noise_std > 0:
                random_noise = tf.random.normal(shape=tf.shape(input=score), mean=0,\
                    stddev=self.noise_std, dtype=score.dtype, seed=self.seed)
                score = score + random_noise

        if self.mode == 'hard':
            probabilities = tf.cast(score > 0, score.dtype)
        else:
            probabilities = tf.sigmoid(score)

        
        attention_weights = _monotonic_attetion(probabilities, prev_attention, self.mode)
        attention_weights = tf.expand_dims(attention_weights, 1)

        #context_vector shape (batch_size, hidden_size)
        context_vector = tf.matmul(attention_weights, encoder_outputs)
        context_vector = tf.squeeze(context_vector, 1, name="context_vector")

        return context_vector, tf.squeeze(attention_weights, 1, name='attention_weights')

    def get_config(self):
        config = {'units': self.units,
                  'mode': self.mode,
                  'scaling_factor': self.scaling_factor,
                  'noise_std': self.noise_std,
                  'weights_initializer': initializers.serialize(self.weights_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'weights_regularizer': self.weights_regularizer 
                  }

        base_config = super(MonotonicBahaAtt, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

### One step decoder

In [ ]:
class OneStepDecoder(tf.keras.Model):

    def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super().__init__()
        
        self.tar_vocab_size = tar_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units

    
    #def build(self, input_shape):

        self.decod_embedd = Embedding(input_dim = self.tar_vocab_size, output_dim = self.embedding_dim,input_length = self.input_length,
                                      weights = [embedding_matrix] , name="embedding_layer_onestepdecoder", mask_zero=True)
        
        self.decod_LSTM = LSTM(units = self.dec_units, return_state=True, activation = 'tanh', 
                               recurrent_activation ='sigmoid', recurrent_dropout = 0, unroll=False,use_bias=True,
                               kernel_regularizer= regularizers.l2(1e-6))
        
        self.MonotonicBahaAtt=MonotonicBahaAtt(self.att_units,
                 mode=self.score_fun,
                 return_aweights=False,
                 scaling_factor=None,
                 noise_std=0,
                 weights_initializer='he_normal',
                 bias_initializer='zeros',)

        self.dense = Dense(units=self.tar_vocab_size)

    def call(self,input_to_decoder, encoder_output, state_h,state_c, attention_weights):
        
        d_embedd = self.decod_embedd(input_to_decoder)

        context_vector,attention_weights = self.MonotonicBahaAtt(state_h,encoder_output, attention_weights)

        d_embedd = tf.concat([tf.expand_dims(context_vector,1), d_embedd], axis=-1)

        d_out, d_hidden, d_cell = self.decod_LSTM(d_embedd, initial_state=[state_h, state_c])

        output = self.dense(d_out)
        #print(d_out.shape)

        return output, d_hidden, d_cell, attention_weights, context_vector

    def get_config(self):

        config = {'tar_vocab_size': self.tar_vocab_size,
                  'embedding_dim': self.embedding_dim,
                  'input_length': self.input_length,
                  'dec_units': self.dec_units,
                  'score_fun': self.score_fun,
                  'att_units': self.att_units
                  }
        
        base_config = super(OneStepDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
       return cls(**config)

### Decoder

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, output_length, dec_units ,score_fun ,att_units):
        super().__init__()

        self.out_vocab_size = out_vocab_size
        self.embedding_dim = embedding_dim
        self.output_length = output_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units


    #def build(self, input_shape):
        self.onestep_decoder = OneStepDecoder(self.out_vocab_size, self.embedding_dim, self.output_length, self.dec_units,
                                              self.score_fun ,self.att_units)

        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state, attention_weights ):

        all_outputs = tf.TensorArray(tf.float32, size=tf.shape(input_to_decoder)[1], name='output_arrays')  #size=input_to_decoder.shape[1]

        #shape = tf.shape(input_to_decoder)[1]
        #input_to_decoder.shape[1]  #range((tf.shape(input_to_decoder)[1]))
        #itr = tf.cast((tf.shape(input_to_decoder)[1]), dtype='int32')
        #itr = input_to_decoder.get_shape().as_list()[1]
        itr = tf.shape(input_to_decoder)[1]

        #_,itr = tf.shape(input_to_decoder)
        #itr = int(itr)

        for tstep in range(itr):    

            output, decoder_hidden_state, decoder_cell_state, attention_weights, context_vector = self.onestep_decoder(
            input_to_decoder[:, tstep:tstep+1], encoder_output, decoder_hidden_state, 
                                 decoder_cell_state, attention_weights)                      #default teacher forcing

            all_outputs = all_outputs.write(tstep, output)
        
        all_outputs = tf.transpose(all_outputs.stack(), [1,0,2])

        return all_outputs

    def get_config(self):
        config = {'out_vocab_size': self.out_vocab_size,
                  'embedding_dim': self.embedding_dim,
                  'output_length': self.output_length,
                  'dec_units': self.dec_units,
                  'score_fun': self.score_fun,
                  'att_units': self.att_units}

        base_config = super(Decoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)
        

## Data pipeline

In [22]:
class Dataset:
    def __init__(self, data, token_perturbation, token_correct, max_len):
        self.encoder_inps = data['encoder_input'].values
        self.decoder_inps = data['decoder_input'].values
        self.decoder_outs = data['decoder_output'].values
        self.token_correct = token_correct
        self.token_perturbation = token_perturbation
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.token_perturbation.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.token_correct.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.token_correct.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='float32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='float32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='float32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)



In [23]:
class Dataloader(tf.keras.utils.Sequence):
    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        
        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

### Model

In [ ]:
class attention_model(tf.keras.Model):
    def __init__(self,e_vocab_size, d_vocab_size, embedding_dim_e,embedding_dim_d, i_length, o_length, enc_units, 
                 dec_units ,score_fun ,att_units):
        super().__init__()

        self.e_vocab_size = e_vocab_size
        self.d_vocab_size = d_vocab_size
        self.embedding_dim_e = embedding_dim_e
        self.embedding_dim_d = embedding_dim_d
        self.i_length = i_length
        self.o_length = o_length
        self.enc_units = enc_units
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units

    #def build(self, input_shape):
        self.encoder = Encoder(self.e_vocab_size,self.embedding_dim_e,self.enc_units,self.i_length)
        self.decoder = Decoder(self.d_vocab_size,self.embedding_dim_d,self.o_length,self.dec_units,self.score_fun,self.att_units)

    def call(self, data, training=True):
        e_input,d_input = data[0], data[1]
        
        e_output, e_hidden, e_cell = self.encoder(e_input)
    
        d_hidden = e_hidden  #initial decoder state is equal to final encoder hidden state
        d_cell = e_cell

        attention_weights = np.zeros((512, 16), dtype='float32')
        attention_weights[:, 0] = 1
        
        final= self.decoder(d_input,e_output,d_hidden,d_cell, attention_weights)
        return final

    def get_config(self):
        
        config={'e_vocab_size': self.e_vocab_size,
                'd_vocab_size': self.d_vocab_size,
                'embedding_dim_e': self.embedding_dim_e,
                'embedding_dim_d': self.embedding_dim_d,
                'i_length': self.i_length,
                'o_length': self.o_length,
                'enc_units': self.enc_units,
                'dec_units': self.dec_units,
                'score_fun': self.score_fun,
                'att_units': self.att_units}
        
        base_config = super(attention_model, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)



## Custom Loss Function

In [24]:
optimizer = tf.keras.optimizers.Adam(clipnorm=1)
#loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Datagenerator

In [25]:
train_dataset = Dataset(train, token_perturbation, token_correct, 16)
validation_dataset  = Dataset(validation, token_perturbation, token_correct, 16)
test_dataset = Dataset(test, token_perturbation, token_correct, 16)

train_dataloader = Dataloader(train_dataset, batch_size=512)
validation_dataloader = Dataloader(validation_dataset, batch_size=512)
test_dataloader = Dataloader(test_dataset, batch_size=512)

print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 16) (512, 16) (512, 16)


***

In [ ]:
pickle.dump(attention_model,open('/content/drive/MyDrive/Project_2/att.pkl','wb'))

In [ ]:
att = pickle.load(open('/content/drive/MyDrive/Project_2/att.pkl','rb'))

## Training

In [ ]:
e_vocab_size = v_perturbation
d_vocab_size = v_correct
embedding_dim_e = 300
embedding_dim_d = 300
i_length = 16
o_length = 16
enc_units = 300
dec_units = 300
score_fun = 'parallel'
att_units = 300

In [ ]:
attention__ = att(e_vocab_size, d_vocab_size, embedding_dim_e,embedding_dim_d, i_length, o_length, enc_units, 
                 dec_units ,score_fun ,att_units)

In [ ]:
attention__ = attention_model(e_vocab_size, d_vocab_size, embedding_dim_e,embedding_dim_d, i_length, o_length, enc_units, 
                 dec_units ,score_fun ,att_units)

In [ ]:
attention__.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

### Callbacks

In [ ]:
import datetime
logfile2 = "/content/drive/MyDrive/Project_2/logs/attention/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#logfile2 = "C:\\Users\\rpris\\Google Drive\\Project_2\\logs\\attention\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tfboard = tf.keras.callbacks.TensorBoard(log_dir=logfile2, histogram_freq=1, write_graph=True)

chkfile2 = "/content/drive/MyDrive/Project_2/wts/attention/weights-{epoch:02d}-{val_loss:.4f}.hdf5"
#chkfile2 = "C:\\Users\\rpris\\Google Drive\\Project_2\\wts\\attention\\weights-{epoch:02d}-{val_loss:.4f}.hdf5"
chkpt =tf.keras.callbacks.ModelCheckpoint(chkfile2, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=0, mode='min')

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='min')

In [ ]:
attention__(tf.ones(shape=(512,512,16)))

In [ ]:
attention__.load_weights("/content/drive/MyDrive/Project_2/wts/attention/weights-21-0.2798.hdf5")
#attention__.load_weights("C:\\Users\\rpris\\Google Drive\\Project_2\\wts\\attention\\mono\\weights-36-0.9240.hdf5")

In [ ]:
train_steps=train.shape[0]//512
valid_steps=validation.shape[0]//512
attention__.fit(train_dataloader, steps_per_epoch=train_steps, epochs=120, validation_data=validation_dataloader,
                         validation_steps=valid_steps, )

In [ ]:
attention__.save('/content/drive/MyDrive/Project_2/Pipeline/attention_model/')

## Inference

In [ ]:
decoder_layer = attention__.layers[1]

In [ ]:
onestep = decoder_layer.layers[0]

In [ ]:
def inference_att(enc_inp):

    enc_inp = enc_inp.replace(",", "")
    enc_inp = enc_inp.replace("?", "")
    enc_inp = enc_inp.replace("!", "")
    translation=""
    
    e_input=[]
    for i in enc_inp.split():
        if token_perturbation.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(token_perturbation.word_index.get(i))
    #e_input = [token_perturbation.word_index[i] for i in enc_inp.split(" ")]
    e_input = pad_sequences([e_input], maxlen=16, padding='post')
    
    
    e_output, e_hidden, e_cell = attention__.layers[0](e_input,0)
    
    d_hidden = e_hidden
    d_cell = e_cell                   #final encoder state is equal to initial decoder state
    
    #initial decoder input is start
    d_input = tf.expand_dims([token_correct.word_index['<start>']],0)

    attention_weights = np.zeros((1,16), dtype='float32')
    attention_weights[:,0] = 1

    for word in range(16):
        
        predicted, d_hidden, d_cell, attention_weights, context_vector = onestep(d_input, e_output, d_hidden, d_cell,
                                                                                 attention_weights)        
        
        #attention_weights = tf.reshape(attention_weights, (-1,))  #creating a 1d tensor attention weights to store them
        #attention_plot[word] = attention_weights.numpy()
        
        #making predictions
        predicted_word_index = tf.argmax(predicted[0]).numpy()  #it will give the highest probability word
      

        #stopping when reaching "<end>"
        if token_correct.index_word[predicted_word_index] == '<end>':
            return translation #, attention_plot
        
        #appending the result with predicted words
        translation = translation + token_correct.index_word[predicted_word_index] + ' '
        
        # the predicted ID is fed back into the model
        d_input = tf.expand_dims([predicted_word_index], 0)
        
    return translation #, attention_plot

In [ ]:
inference_att('i have an pen')

'i have a pen '

In [ ]:
sample_train = train.sample(1000)
result = []
for enc_inp,dec_inp,_ in sample_train.values:
    pred = inference_att(enc_inp)
    result.append(pred)
sample_train['correct_output'] = sample_train['decoder_output']
sample_train['predicted_output'] = result
sample_train = sample_train.drop(['decoder_input', 'decoder_output'], axis=1)
sample_train.head(10)

,encoder_input,correct_output,predicted_output
185429,what an way to finish,what a way to finish <end>,what a way to finish
180952,you wanna have drink before we start dancing,you wanna have a drink before we start dancing...,you wanna have a drink before we start dancing
100305,imagine an important part of my life wasted be...,imagine an important part of my life wasted be...,imagine an important part of my life wasted be...
171689,all way to urbana for a one night stand,all the way to urbana for a one night stand <...,all the way to urbana for a one night
31490,can we leave out an bowl of milk mommy,can we leave out a bowl of milk mommy <end>,can we leave out a bowl of milk mommy
63907,i am assuming that is what made him dead one,i am assuming that is what made him the dead o...,i am assuming that is what made him a dead one
143783,what waste of time,what a waste of time <end>,what a waste of time
158086,that why you brought him,is that why you brought him <end>,that is why you brought him
75998,but i have got weak stomach,but i have got a weak stomach <end>,but i have got a weak stomach
169482,is david dunne their,is david dunne there <end>,is david dunne


In [ ]:
sample_validation = validation.sample(1000)
result = []
for enc_inp,dec_inp,_ in sample_validation.values:
    pred = inference_att(enc_inp)
    result.append(pred)
sample_validation['correct_output'] = sample_validation['decoder_output']
sample_validation['predicted_output'] = result
sample_validation = sample_validation.drop(['decoder_input', 'decoder_output'], axis=1)
sample_validation.head(10)

,encoder_input,correct_output,predicted_output
193097,every so often they imagine story good enough ...,every so often they imagine a story good enoug...,every so often they imagine the story good eno...
201723,out of mouths of babes louis,out of the mouths of babes louis <end>,out of the trees of crackers
155259,whatever happened was accident,whatever happened was an accident <end>,whatever was happened was an accident
123156,christ i did not say you were out of game,christ i did not say you were out of the game ...,christ i did not say you were out of the game
184780,that your game,is that your game <end>,that is your game
12080,in an manner of speaking,in a manner of speaking <end>,in a manner of speaking
190064,well it certainly is coincidence,well it certainly is a coincidence <end>,well it is certainly a coincidence
159590,i am sorry it was an accident,i am sorry it was an accident <end>,i am sorry it was an accident
484,dursleys ducked but harry leapt into air tryi...,the dursleys ducked but harry leapt into the a...,the dursleys he wins harry into the supplies d...
103312,the girl is name doreen,the girl is name was doreen <end>,the girl is name was


***

In [ ]:
sample_test = test.sample(1000)
result = []
for enc_inp,dec_inp,_ in sample_test.values:
    pred = inference_att(enc_inp)
    result.append(pred)
sample_test['correct_output'] = sample_test['decoder_output']
sample_test['predicted_output'] = result
sample_test = sample_test.drop(['decoder_input', 'decoder_output'], axis=1)
sample_test.head(10)

,encoder_input,correct_output,predicted_output
164628,you look like you were very sensitive child,you look like you were a very sensitive child ...,you look like you were a very sensitive child
194417,and main reason ellie is that way is because y...,and the main reason ellie is that way is becau...,the reason ellie is that goes the all cause yo...
148272,so what you doing later,so what are you doing later <end>,so what are you doing later
42984,you never played violin,you never played a violin <end>,you never played the violin
186169,fifty will get you hundred that he did not do it,fifty will get you a hundred that he did not d...,fifty will get you a hundred that he did not d...
29290,i mean yeah i would say this an all right shirt,i mean yeah i would say this was an all right ...,i mean yeah i would say this is all right an s...
107313,they said hello,he said hello <end>,he said hello
11459,they fucked around with it and torah melted th...,they fucked around with it and the torah melte...,they fucked around with it and a tarp their fa...
198668,you looking for catherine not me,you are looking for catherine not me <end>,you are looking for catherine not me
100080,we shoulda been their to back you up,we shoulda been there to back you up <end>,we shoulda been there to back you


## BLEU Score

In [ ]:
import nltk.translate.bleu_score as bleu

In [ ]:
train_bleu = []
for enc_inp,correct,pred in sample_train.values:
    
    correct = correct.split()[:-1]    #removing end
    pred = pred.split()
    
    if len(correct) == len(pred):
        train_bleu.append(bleu.sentence_bleu([correct],pred))
        
print("BLEU Score of train dataset is",sum(train_bleu)/len(train_bleu))

BLEU Score of train dataset is 0.978379354270412


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


<font size=4 color='red'> BLEU Score of train dataset is 0.9948193553886482

In [ ]:
validation_bleu = []
for enc_inp,correct,pred in sample_validation.values:
    
    correct = correct.split()[:-1]    #removing end
    pred = pred.split()

    if len(correct) == len(pred):
        validation_bleu.append(bleu.sentence_bleu([correct],pred))
        
print("BLEU Score of validation dataset is",sum(validation_bleu)/len(validation_bleu))

BLEU Score of validation dataset is 0.8905224236711871


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


<font size=4 color='red'> BLEU Score of validation dataset is 0.8905224236711871



In [ ]:
test_bleu = []
for enc_inp,correct,pred in sample_test.values:
    
    correct = correct.split()[:-1]    #removing end
    pred = pred.split()

    if len(correct) == len(pred):
        test_bleu.append(bleu.sentence_bleu([correct],pred))
        
print("BLEU Score of test dataset is",sum(test_bleu)/len(test_bleu))

BLEU Score of test dataset is 0.899345597466305


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


<font size=4 color='red'> BLEU Score of test dataset is 0.899345597466305

## Error Analysis

<font size=4, color='red'> 1. Many words and sentence in our corpus are not normal, meaning they are in  movie language so embedding matrix returns 0 for those words as they are  incorrect in proper english.  [This is a major reason]</font>  
__Ex.__  
ui amu the director uiu make the casting decis.  
we shoulda been their to back you up	
yer doing whacha
i enclose a representation of mbwun  
wut up
***

<font size=4, color="red">2. confusion between whether to add "a" , "an"  or replace them with "the" </font>  
__Ex.__  
incorrect-->you never played violin  
correct-->you never played a violin  
predicted--> you never played the violin  

**_Note: Happens in a few cases only_**
***

<font size=4, color='red'>3. Gets confused while dealing with names because there are not enough sentence which contains same name. </font>  
__Ex.__  
incorrect-->japan four thousand miles away  
correct-->japan is four thousand miles away  
pred-->steveston is eight thousand miles 

***

<font size=4, color='red'> 4. questions starting with 'is' 'are' are converted to affirmative sentence because it was trained on "is" "are" as perturbations.  </font>  
__Ex.__  
incorrect-->that your game  
correct-->is that your game  
predicted-->that is your game  

**_Note: Happens in a few cases only_**